# 2-1 物体検出とは
## 物体検出の概要
物体検出は画像中の複数の物体に対して，物体の領域と物体名を特定するタスクを指す．
物体の位置を示す枠をバウンディングボックス（Bounding Box）と呼び，バウンディングボックス内の物体名と，その確信度（confidence）を同時に出力する．
物体検出の結果の一例を次に示す．

<img src="../image/IMG_0186.jpg" alt="outputs_ssd">

## 物体検出タスクのインプットとアウトプット
物体検出のインプットは画像で，次の3種類の情報を出力する．

1. バウンディングボックスの位置と大きさの情報
1. 各バウンディングボックスの囲む物体のラベル情報
1. 検出した物体の信頼度  

1つめのバウンディングボックスの位置と大きさの情報は，次に示すように2通りの表現方法がある．

<img src="../image/IMG_0187.jpg" alt="representation_bounding_box">

- 長方形の4つの頂点の座標  
左端の$x_{min}$，上端の$y_{min}$，右端の$x_{max}$，下端の$y_{max}$でバウンディングボックスを与える．
- 長方形の中心座標と辺の長さ  
長方形の中心$(cx, cy)$と縦と横の長さ$h, w$でバウンディングボックスを与える．

ラベルの情報は検出したい物体のクラス数$O$に背景クラス（background）を足した$(O+1)$種類のクラスから各バウンディングボックスにつき1つが割り当てられ，最終的に信頼度が高いバウンディングボックスのみがアウトプットとなる．

## VOC データセット
VOC データセットはヨーロッパの研究コミュニティ PASCAL が作成し，コンテストで使用されているデータセットである．
主に2007年と2012年のデータセットが使用されるが，ここでは VOC2012 を用いる．
20種類のクラスからなる，5,717枚の訓練データと5,823枚の検証データを含む．
20種類のクラスは aeroplane, bicycle, bird, boat などがあり，これに背景クラスを加えた21クラスで実験を行う．  
データセットの各画像には，画像の左上を$(1,1)$としてバウンディングボックスの4つの頂点と物体のラベルを格納した xml ファイルが提供されている．

## SSD による物体検出の流れ
SSD には 入力画像を300x300にリサイズして入力する SSD300 と512x512で処理する SSD512 があるが，ここでは SSD300 の方を扱う．
下の図に示すように，SSDでは定型的な長方形をデフォルトボックスとして用意しておき，そのデフォルトボックスをどのように変形させればバウンディングボックスになるかを示すオフセット情報を出力する．

デフォルトボックスが$(cx_d, cy_d, w_d, h_d)$，オフセット情報が$(\Delta cx, \Delta cy, \Delta w, \Delta h)$で与えられたとき，バウンディングボックスは次のように与えられる．

$$
\begin{align}
cx &= cx_d(1+0.1 \Delta cx) \\
cy &= cy_d(1+0.1 \Delta cy) \\
w &= h_d \times exp(0.2 \Delta w) \\
h &= w_d \times exp(0.2 \Delta h)
\end{align}
$$

<img src="../image/IMG_0188.jpg" alt="default_box">

SSD による物体検出の各ステップを下の図に示す．

<img src="../image/IMG_0189.jpg" alt="ssd_steps">

1. 入力画像のリサイズと標準化  
入力画像を300x300にリサイズし，色情報を標準化する
1. デフォルトボックスの準備
入力画像にかかわらず様々な大きさとアスペクト比を持つ8,732個のデフォルトボックスを準備する．
1. SSD に画像を入力し各情報を出力  
各デフォルトボックスに対して21種類のラベル情報と4つのオフセット情報を出力する．合計の出力数は $8732 \times (21 + 4) = 218300$ 個となる．
1. 信頼度の高いデフォルトボックスを取り出す
上で求めたデフォルトボックスのうち，信頼度でランキングして top_k（ここでは200）個を取り出す．
1. バウンディングボックスへの変換と重複の削除
top_k 個のデフォルトボックスをバウンディングボックスに変換する．重複の大きいバウンディングボックスがある場合には，最も信頼度が高いものだけを取り出す．
1. バウンディングボックスとラベルを出力する
信頼度について閾値を定め，閾値以上の信頼度を持つバウンディングボックスのみを出力する．